In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "Informer-Tensorflow/models")
sys.path.insert(1, "Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=8)
    x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()

[TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32])]


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(8, 2000, 1)]            0         
_________________________________________________________________
dense (Dense)                (8, 2000, 32)             64        
_________________________________________________________________
positional_embedding (Positi (8, 2000, 32)             0         
_________________________________________________________________
multi_head_prob_attention (M (8, 2000, 64)             10496     
_________________________________________________________________
conv_layer (ConvLayer)       (8, 666, 32)              6304      
_________________________________________________________________
layer_normalization (LayerNo (8, 666, 32)              64        
_________________________________________________________________
global_average_pooling1d (Gl (8, 32)                   0     

In [9]:
x = tf.ones([8,2000,1])
model(x)

<tf.Tensor: shape=(8, 5), dtype=float32, numpy=
array([[ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908],
       [ 0.92995197, -1.2726281 , -0.37056383, -2.17059   , -0.35434908]],
      dtype=float32)>

In [10]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [11]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [12]:
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [15]:
model.fit(x=X[0:904], y=Y[0:904], batch_size=8, epochs=500, validation_data=(X[904:], Y[904:]),
         callbacks=[tensorboard_callback])

Epoch 1/500
113/113 [==============================] - 137s 1s/step - loss: 0.4059 - accuracy: 0.8595 - val_loss: 2.5529 - val_accuracy: 0.4896
Epoch 2/500
113/113 [==============================] - 7s 58ms/step - loss: 0.4040 - accuracy: 0.8772 - val_loss: 13.8416 - val_accuracy: 0.2083
Epoch 3/500
113/113 [==============================] - 7s 58ms/step - loss: 0.3946 - accuracy: 0.8717 - val_loss: 2.9818 - val_accuracy: 0.5104
Epoch 4/500
113/113 [==============================] - 7s 58ms/step - loss: 0.4141 - accuracy: 0.8606 - val_loss: 1.8517 - val_accuracy: 0.5417
Epoch 5/500
113/113 [==============================] - 7s 58ms/step - loss: 0.4007 - accuracy: 0.8684 - val_loss: 5.6026 - val_accuracy: 0.4167
Epoch 6/500
113/113 [==============================] - 7s 59ms/step - loss: 0.4017 - accuracy: 0.8595 - val_loss: 3.4229 - val_accuracy: 0.4688
Epoch 7/500
113/113 [==============================] - 7s 65ms/step - loss: 0.4007 - accuracy: 0.8650 - val_loss: 4.4034 - val_accuracy

113/113 [==============================] - 7s 63ms/step - loss: 0.3902 - accuracy: 0.8761 - val_loss: 6.8153 - val_accuracy: 0.2188
Epoch 57/500
113/113 [==============================] - 9s 77ms/step - loss: 0.3870 - accuracy: 0.8717 - val_loss: 1.6397 - val_accuracy: 0.5938
Epoch 58/500
113/113 [==============================] - 7s 63ms/step - loss: 0.3942 - accuracy: 0.8662 - val_loss: 4.3270 - val_accuracy: 0.4375
Epoch 59/500
113/113 [==============================] - 7s 64ms/step - loss: 0.3966 - accuracy: 0.8673 - val_loss: 2.8714 - val_accuracy: 0.2604
Epoch 60/500
113/113 [==============================] - 7s 60ms/step - loss: 0.4033 - accuracy: 0.8606 - val_loss: 6.1249 - val_accuracy: 0.2500
Epoch 61/500
113/113 [==============================] - 7s 61ms/step - loss: 0.4035 - accuracy: 0.8662 - val_loss: 3.2466 - val_accuracy: 0.5312
Epoch 62/500
113/113 [==============================] - 7s 61ms/step - loss: 0.4070 - accuracy: 0.8684 - val_loss: 0.8848 - val_accuracy: 0.718

113/113 [==============================] - 7s 60ms/step - loss: 0.3778 - accuracy: 0.8750 - val_loss: 6.1609 - val_accuracy: 0.1979
Epoch 113/500
113/113 [==============================] - 7s 61ms/step - loss: 0.3987 - accuracy: 0.8551 - val_loss: 2.2957 - val_accuracy: 0.4271
Epoch 114/500
113/113 [==============================] - 7s 60ms/step - loss: 0.3791 - accuracy: 0.8662 - val_loss: 4.1937 - val_accuracy: 0.4896
Epoch 115/500
113/113 [==============================] - 7s 61ms/step - loss: 0.3756 - accuracy: 0.8772 - val_loss: 1.2549 - val_accuracy: 0.5521
Epoch 116/500
113/113 [==============================] - 7s 60ms/step - loss: 0.3807 - accuracy: 0.8628 - val_loss: 6.4978 - val_accuracy: 0.4896
Epoch 117/500
113/113 [==============================] - 9s 64ms/step - loss: 0.3724 - accuracy: 0.8805 - val_loss: 1.1148 - val_accuracy: 0.6979
Epoch 118/500
113/113 [==============================] - 7s 61ms/step - loss: 0.4000 - accuracy: 0.8684 - val_loss: 1.4490 - val_accuracy:

KeyboardInterrupt: 